In [24]:
#Imports
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import numpy as np

In [25]:
# Watermark
print('Nathan Young\nJunior Data Analyst\nCenter for the Study of Free Enterprise')
%load_ext watermark
%watermark -a "Western Carolina University" -u -d -p pandas

Nathan Young
Junior Data Analyst
Center for the Study of Free Enterprise
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Western Carolina University 
last updated: 2020-01-28 

pandas 0.25.3


In [26]:
# Create backups
df_backup = pd.read_csv('./Updates/STG_FRED_Homeownership_Rate_by_County.txt')
df_backup.to_csv('./Backups/STG_FRED_Homeownership_Rate_by_County_BACKUP.txt')

In [27]:
# Getting and reading new data 
df = pd.read_excel("https://geofred.stlouisfed.org/api/download.php?theme=pubugn&colorCount=5&reverseColors=false&intervalMethod=fractile&displayStateOutline=true&lng=-89.96&lat=40.81&zoom=4&showLabels=true&showValues=true&regionType=county&seriesTypeId=157125&attributes=Not+Seasonally+Adjusted%2C+Annual%2C+Rate&aggregationFrequency=Annual&aggregationType=Average&transformation=lin&date=2017-01-01&type=xls&startDate=2009-01-01&endDate=2017-01-01&mapWidth=999&mapHeight=521&hideLegend=false", skiprows=1)
df.head(2)

,Series ID,Region Name,Region Code,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,HOWNRATEACS045001,"Abbeville County, SC",45001,82.305188,80.790340,79.586980,78.165868,76.560964,78.204917,79.132274,78.025359,77.285284
1,HOWNRATEACS022001,"Acadia Parish, LA",22001,71.514505,70.396228,69.800876,69.941320,71.363091,70.431491,72.129833,72.399002,72.378770


In [28]:
# Filter data to display only North Carolina
filter1 = df['Region Name'].str.contains(', NC')
df_nc = df[filter1]
df_nc.head(2)

,Series ID,Region Name,Region Code,2009,2010,2011,2012,2013,2014,2015,2016,2017
24,HOWNRATEACS037001,"Alamance County, NC",37001,69.825326,68.848831,69.065412,67.489438,67.142423,66.953034,66.480499,65.752829,66.627537
35,HOWNRATEACS037003,"Alexander County, NC",37003,80.944634,79.988556,79.256559,78.341356,78.471513,79.132140,77.574326,75.616271,74.305381


In [29]:
# Set Index to Series ID
df_nc.set_index(df_nc['Series ID'], inplace = True)
df_nc.head(2)

,Series ID,Region Name,Region Code,2009,2010,2011,2012,2013,2014,2015,2016,2017
Series ID,,,,,,,,,,,,
HOWNRATEACS037001,HOWNRATEACS037001,"Alamance County, NC",37001,69.825326,68.848831,69.065412,67.489438,67.142423,66.953034,66.480499,65.752829,66.627537
HOWNRATEACS037003,HOWNRATEACS037003,"Alexander County, NC",37003,80.944634,79.988556,79.256559,78.341356,78.471513,79.132140,77.574326,75.616271,74.305381


In [30]:
# Drop Series ID column
df_nc.drop('Series ID', axis = 1, inplace = True)
df_nc.head(2)

,Region Name,Region Code,2009,2010,2011,2012,2013,2014,2015,2016,2017
Series ID,,,,,,,,,,,
HOWNRATEACS037001,"Alamance County, NC",37001,69.825326,68.848831,69.065412,67.489438,67.142423,66.953034,66.480499,65.752829,66.627537
HOWNRATEACS037003,"Alexander County, NC",37003,80.944634,79.988556,79.256559,78.341356,78.471513,79.132140,77.574326,75.616271,74.305381


In [31]:
# Save file to tab delimited txt for upload to SSMS
df_nc.to_csv('./Updates/STG_FRED_Homeownership_Rate_by_County.txt', sep = '\t')

In [32]:
#Reset Index for upload to database
df_nc = df_nc.reset_index()    

In [33]:
column_list = df_nc.columns.values
for i in column_list:
    df_nc.loc[df_nc[i].isnull(),i]=0

In [34]:
#Connect to database and create cursor
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=TITANIUM-BOOK;'
                      'Database=DataDashboard;'
                      'Trusted_Connection=yes;',
                    autocommit=True)

c = con.cursor()

In [35]:
#Drop old backup table
c.execute('drop table STG_FRED_Homeownership_Rate_by_County_BACKUP')

In [36]:
#Create new backup
c.execute('''sp_rename 'dbo.STG_FRED_Homeownership_Rate_by_County','STG_FRED_Homeownership_Rate_by_County_BACKUP';''')

In [37]:
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON


SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_FRED_Homeownership_Rate_by_County](
	[Series ID] [varchar](14) NULL,
	[Region Name] [varchar](23) NULL,
	[Region Code] [int] NULL,
	[2009] [float] NULL,
	[2010] [float] NULL,
	[2011] [float] NULL,
	[2012] [float] NULL,
	[2013] [float] NULL,
	[2014] [float] NULL,
	[2015] [float] NULL,
	[2016] [float] NULL,
	[2017] [float] NULL,
	[2018] [float] NULL,
    [2019] [float] NULL,
    [2020] [float] NULL,
    [2021] [float] NULL,
    [2022] [float] NULL,
    [2023] [float] NULL,
    [2024] [float] NULL,
    [2025] [float] NULL
) ON [PRIMARY]''')

In [38]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#df: pandas.dataframe; mTableName:table name in MS SQL
#warning: discard old table if exists
df_nc.to_sql('STG_FRED_Homeownership_Rate_by_County', con=engine, if_exists='replace', index=False)